In [1]:
print("hello world")

hello world


In [1]:

import pandas

In [7]:
df = pandas.read_csv("../data/data/benin-malanville.csv")

In [6]:
df.head()

,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-08-09 00:01,-1.2,-0.2,-1.1,0.0,0.0,26.2,93.4,0.0,0.4,0.1,122.1,0.0,998,0,0.0,26.3,26.2,NaN
1,2021-08-09 00:02,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.6,0.0,0.0,0.0,0.0,0.0,998,0,0.0,26.3,26.2,NaN
2,2021-08-09 00:03,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.7,0.3,1.1,0.5,124.6,1.5,997,0,0.0,26.4,26.2,NaN
3,2021-08-09 00:04,-1.1,-0.1,-1.0,0.0,0.0,26.2,93.3,0.2,0.7,0.4,120.3,1.3,997,0,0.0,26.4,26.3,NaN
4,2021-08-09 00:05,-1.0,-0.1,-1.0,0.0,0.0,26.2,93.3,0.1,0.7,0.3,113.2,1.0,997,0,0.0,26.4,26.3,NaN


## Data Quality Checks
We will now check for missing values, outliers, and incorrect entries in the following columns:
- GHI, DNI, DHI (solar irradiance)
- ModA, ModB (sensor readings)
- WS, WSgust (wind speed data)

In [9]:
# Check for missing values in key columns
columns_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
missing_values = df[columns_to_check].isnull().count()
print('Missing values per column:')
print(missing_values)

Missing values per column:
GHI       525600
DNI       525600
DHI       525600
ModA      525600
ModB      525600
WS        525600
WSgust    525600
dtype: int64


In [10]:
# Check for outliers using the IQR method
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()
outliers = {col: detect_outliers_iqr(df[col].dropna()) for col in columns_to_check}
print('Outlier count per column:')
print(outliers)

Outlier count per column:
{'GHI': np.int64(145), 'DNI': np.int64(15196), 'DHI': np.int64(8733), 'ModA': np.int64(98), 'ModB': np.int64(240), 'WS': np.int64(6717), 'WSgust': np.int64(5368)}


In [11]:
# Check for negative or obviously incorrect values
incorrect_entries = {}
for col in ['GHI', 'DNI', 'DHI', 'ModA', 'ModB']:
    incorrect_entries[col] = (df[col] < 0).sum()
for col in ['WS', 'WSgust']:
    incorrect_entries[col] = (df[col] < 0).sum() + (df[col] > 100).sum()  # wind speed >100 m/s is likely incorrect
print('Incorrect entry count per column:')
print(incorrect_entries)

Incorrect entry count per column:
{'GHI': np.int64(258847), 'DNI': np.int64(275987), 'DHI': np.int64(259182), 'ModA': np.int64(0), 'ModB': np.int64(0), 'WS': np.int64(0), 'WSgust': np.int64(0)}
